<a href="https://colab.research.google.com/github/Wazhee/Stroke-Prediction/blob/main/Stroke_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load Dependencies

In [ ]:
from google.colab import drive
drive.mount('/content/MyDrive')

Mounted at /content/MyDrive


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.layers import *
from keras.models import *
from keras.preprocessing import image
import pandas as pd
import os
from tqdm import tqdm
import cv2
import tensorflow as tf

# Load CT Images

In [ ]:
df, data = pd.read_csv('/content/MyDrive/MyDrive/metadata.csv'), dict()
W, H = 512, 512 # correct dimensions
st_final = list(df['Post_ST_3M_Final'])
patient_ids = list(df['Patient_code'])
gt, images = [], []

# load images
path, data = '/content/MyDrive/MyDrive/PNG CT images/', dict()
directory = os.listdir(path)
directory.remove('.DS_Store')
directory = [pre for pre in directory if 'Pre' in pre]


# load all pre stroke images
for subject in tqdm(directory):
    idx = int(subject.split('_')[0])
    imgpath = path+subject
    # load all images for subject
    folder = os.listdir(imgpath)
    folder = [item for item in folder if '.png' in item]
    for count in range(len(folder)):
        gt.append(st_final[patient_ids.index(idx)])
        img = cv2.imread(f'{imgpath}/{count+1}.png', cv2.IMREAD_GRAYSCALE)
        if (img.shape != (W, H)):
            img = cv2.resize(img, (W,H), cv2.INTER_CUBIC)
        images.append(img)

 33%|███▎      | 14/43 [12:48<24:04, 49.82s/it]

# Create Train and Validation

In [ ]:
train_folderA, train_folderB = './train/stroke/', './train/non_stroke/'
val_folderA, val_folderB = './val/stroke/', './val/non_stroke/'

if not os.path.exists(train_folderA):
    os.makedirs(train_folderA)
if not os.path.exists(train_folderB):
    os.makedirs(train_folderB)
if not os.path.exists(val_folderA):
    os.makedirs(val_folderA)
if not os.path.exists(val_folderB):
    os.makedirs(val_folderB)


size = len(images)
split = int(.8 * size)
for count in tqdm(range(size)):
    if count > split:
        if(gt[count] == 1):
            cv2.imwrite(f'{val_folderA}{count+1}.png', images[count])
        else:
            cv2.imwrite(f'{val_folderB}{count+1}.png', images[count])
    else:
        if(gt[count] == 1):
            cv2.imwrite(f'{train_folderA}{count+1}.png', images[count])
        else:
            cv2.imwrite(f'{train_folderB}{count+1}.png', images[count])

# Model Architecture

In [ ]:
# First Layer
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(512, 512, 3)))
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
# Second Layer
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
# Third Layer
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
# Output Layer
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# Calculate Losss
model.compile(loss=keras.losses.binary_crossentropy, optimizer='adam', metrics=['accuracy'])

# Load Train and Validation Dataset

In [ ]:
train_datagen = image.ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
)

test_dataset = image.ImageDataGenerator(rescale=1./255)

In [ ]:
train_folder, val_folder = './train/', './val/'

train_generator = train_datagen.flow_from_directory(
    train_folder,
    target_size=(W,H),
    batch_size = 32,
    class_mode = 'binary'
)
validation_generator = test_dataset.flow_from_directory(
    val_folder,
    target_size = (512,512),
    batch_size = 32,
    class_mode = 'binary'
)

print(train_generator.class_indices)
print(validation_generator.class_indices)

# Training

In [ ]:
hist = model.fit_generator(
    train_generator,
    steps_per_epoch = 10,
    epochs = 50,
    validation_data = validation_generator,
    validation_steps=2
)